In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Parcial 2016 2do Cuatrimestre, Ejercicio 2 

En este ejercicio queremos programar un sistema que recomiende
textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
es un string de longitud variable. 

Además contamos con un RDD que
indica qué términos le gustan o no a cada usuario de la forma (userId,
término, score) por ejemplo (23, “calesita”, -2). 

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para
los términos que aparecen en el documento. 

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en
estos casos simplemente los consideramos neutros (score=0)

In [2]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [3]:
documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [4]:
# generamos algo del tipo (word, docId) donde word sera nuestra key
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()]).collect()

[('pablo', 1),
 ('honey', 1),
 ('the', 2),
 ('bends', 2),
 ('ok', 3),
 ('computer', 3),
 ('kid', 4),
 ('a', 4),
 ('amnesiac', 5),
 ('hail', 6),
 ('to', 6),
 ('the', 6),
 ('thief', 6),
 ('in', 7),
 ('rainbows', 7),
 ('the', 8),
 ('king', 8),
 ('of', 8),
 ('limbs', 8),
 ('a', 9),
 ('moon', 9),
 ('shaped', 9),
 ('pool', 9)]

In [5]:
# llevamos scores a una representacion que nos permita unir la informacion
scoresRDDForJoin = scoresRDD.map(lambda a: (a[1],(a[0],a[2])))
scoresRDDForJoin.collect()

[('pablo', ('thom', 1)),
 ('honey', ('thom', 1)),
 ('pablo', ('martin', -1)),
 ('honey', ('martin', -1)),
 ('ok', ('martin', 30)),
 ('computer', ('martin', 30))]

In [8]:
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .collect()

[('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('computer', (3, ('martin', 30))),
 ('ok', (3, ('martin', 30))),
 ('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1)))]

In [6]:
# para poder realizar una acumulacion debemos llevar la informacion a la representacion (docId, userId, score) 
# y luego realizar una acumulacion de scores
# ignorando el score

# notese que el K es docId, userId y el V es el score que aporta el termino

documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .collect()

[((1, 'thom'), 1),
 ((1, 'martin'), -1),
 ((3, 'martin'), 30),
 ((3, 'martin'), 30),
 ((1, 'thom'), 1),
 ((1, 'martin'), -1)]

In [6]:
# acumulamos haciendo reduce
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .reduceByKey(lambda a,b: a + b)\
    .collect()

[((3, 'martin'), 60), ((1, 'thom'), 2), ((1, 'martin'), -2)]

In [7]:
# llevamos a la representacion pedida
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .reduceByKey(lambda a,b: a + b)\
    .map(lambda a: (a[0][0], a[0][1], a[1]))\
    .collect()

[(3, 'martin', 60), (1, 'thom', 2), (1, 'martin', -2)]

# Parcial 2016 2do Cuatrimestre, Ejercicio 1

Contamos con un cluster que tiene 4 computadoras. 

Queremos aprovechar el paralelismo del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la
criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo (2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son
mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego
eliminamos todos los mayores a 3 divisibles por 3 y nos queda
(2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así
sucesivamente. 

El resultado final es una lista de números que son
primos. 

Programar este programa usando PySpark

In [29]:
# criterio
# En primer lugar hay que crear un RDD con todos los números y hacer un parallelize con 4 particiones. 
# Luego la resolución pasa por un ciclo en el cual en cada paso del ciclo se obtenga 
# como pivote el mínimo número del RDD que sea mayor al número
# usado como pivote anteriormente, luego simplemente hay que 
# filtrar el RDD eliminando los números que son múltiplos del pivote

In [8]:
type(sc)

pyspark.context.SparkContext

### Verificacion de multiplo

In [67]:
def is_not_multiple(pivot, value):
        if pivot == value:
            return True
        
        if (value % pivot) > 0:
            return True
        else:
            return False
        

In [69]:
assert is_not_multiple(3, 1)
assert is_not_multiple(3, 2)
assert is_not_multiple(3, 3)
assert is_not_multiple(3, 4)
assert is_not_multiple(3, 5)
assert not is_not_multiple(3, 6)
assert is_not_multiple(3, 7)
assert is_not_multiple(3, 8)
assert not is_not_multiple(3, 9)
assert is_not_multiple(3, 10)
assert is_not_multiple(3, 11)
assert not is_not_multiple(3, 12)

assert is_not_multiple(2, 7)
assert is_not_multiple(3, 7)
assert is_not_multiple(5, 7)

In [70]:
# hacemos un caso de juguete de 2 a 20
numbers = sc.parallelize(range(2,21), 4)
pivot = 2

numbers.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

### Caso Base

In [76]:
numbers.filter(lambda a: is_not_multiple(2, a))\ # Filtro (o sea me quedo) con los que no son multiplos de 2.
       .filter(lambda a: is_not_multiple(3, a))\ # Filtro (o sea me quedo) con los que no son multiplos de 3.
       .filter(lambda a: is_not_multiple(5, a)).collect() # Filtro (o sea me quedo) con los que no son multiplos de 5.

[2, 3, 5, 7, 11, 13, 17, 19]

In [83]:
# hacemos un caso de juguete de 2 a 2000
numbers = sc.parallelize(range(2,2000), 4)

pivot = 2

while ((pivot * pivot) <= 2000):
    new_pivot = numbers.filter(lambda a: a > pivot).reduce(lambda a, b: a if a < b else b)
    numbers = numbers.filter(lambda a: is_not_multiple(pivot, a)).cache()
    pivot = new_pivot

In [84]:
numbers.collect()

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541,
 547,
 557,
 563,
 569,
 571,
 577,
 587,
 593,
 599,
 601,
 607,
 613,
 617,
 619,
 631,
 641,
 643,
 647,
 653,
 659,
 661,
 673,
 677,
 683,
 691,
 701,
 709,
 719,
 727,
 733,
 739,
 743,
 751,
 757,
 761,
 769,
 773,
 787,
 797,
 809,
 811,
 821,
 823,
 827,
 829,
 839,
 853,
 857,
 859,
 863,
 877,
 881,
 883,
 887,
 907,
 911,
 919,
 929,
 937,
 941,
 947,
 953,
 967,
 971,
 977,
 983,
 991,
 997,
 1009,
 1013,
 1019,
